# Feature Importance Analysis (Gradient Boosting)

## Goal: Identify Which Features Actually Matter

**NOT for accuracy - FOR FEATURE RANKING!**

**Strategy:**
1. Extract ALL candidate features from raw data
2. Train Gradient Boosting model (FAST: 20 trees, 3-5 min)
3. Analyze feature importance scores
4. Identify top 5-10 features that actually contribute
5. Discard the rest

**Features to Extract:**
1. Competitor count at 1km, 2km, 5km
2. Nearest competitor distance
3. Anchor POI counts (mall, office, transport)
4. Income level by district
5. District density
6. Demand-supply ratio
7. Residential/school/hospital counts

**Output:**
- Feature importance ranking (sorted CSV)
- Top 10 features chart
- Recommendation: which features to keep for final model

---

## Setup

In [ ]:
!pip install -q scikit-survival
print("✓ Installation complete!")

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from shapely.strtree import STRtree
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported!")

## Configuration

In [ ]:
# Target category
TARGET_CATEGORY = 'restaurant'

# Paths
try:
    DATASET_PATH = "/kaggle/input/jakarta-clean-categorized/jakarta_clean_categorized.csv"
    OUTPUT_DIR = Path("/kaggle/working")
    print("✓ Running on Kaggle")
except:
    DATASET_PATH = "data/jakarta_clean_categorized.csv"
    OUTPUT_DIR = Path("outputs")
    OUTPUT_DIR.mkdir(exist_ok=True)
    print("✓ Running locally")

# Gradient Boosting config (OPTIMIZED FOR SPEED)
GB_CONFIG = {
    'n_estimators': 20,      # FAST (just for feature importance)
    'learning_rate': 0.2,
    'max_depth': 3,
    'subsample': 0.8,
    'random_state': 42,
    'verbose': 1
}

print(f"\nTarget: {TARGET_CATEGORY}")
print(f"Gradient Boosting: {GB_CONFIG['n_estimators']} trees (for feature ranking only)")
print(f"\n⚠ NOT optimizing for accuracy - ONLY for feature importance!")

## Load Data

In [ ]:
print("Loading dataset...")
df = pd.read_csv(DATASET_PATH)
print(f"✓ Loaded {len(df):,} POIs total")

# Convert ALL to GeoDataFrame (we need all POIs for context)
gdf_all = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs='EPSG:4326'
).to_crs(epsg=32748)

print(f"✓ Converted all POIs to UTM (EPSG:32748)")

# Filter target category for analysis
gdf_target = gdf_all[gdf_all['poi_type'] == TARGET_CATEGORY].copy()
print(f"✓ Target category: {len(gdf_target):,} {TARGET_CATEGORY}s")

## Create Survival Labels

In [ ]:
print("Creating survival labels...\n")

# Parse dates
gdf_target['date_created_parsed'] = pd.to_datetime(gdf_target['date_created'], errors='coerce')
gdf_target['date_closed_parsed'] = pd.to_datetime(gdf_target['date_closed'], errors='coerce')

REFERENCE_DATE = pd.Timestamp('2024-01-01')
OBSERVATION_WINDOW_DAYS = 365 * 3

# Event and duration
gdf_target['event'] = gdf_target['date_closed_parsed'].notna()
gdf_target['duration'] = np.where(
    gdf_target['event'],
    (gdf_target['date_closed_parsed'] - gdf_target['date_created_parsed']).dt.days,
    (REFERENCE_DATE - gdf_target['date_created_parsed']).dt.days
)

# Categorical label
gdf_target['categorical_label'] = 2  # too new
gdf_target.loc[
    (gdf_target['date_created_parsed'] <= REFERENCE_DATE - timedelta(days=OBSERVATION_WINDOW_DAYS)) & 
    (gdf_target['date_closed_parsed'].notna()) & 
    (gdf_target['date_closed_parsed'] <= REFERENCE_DATE),
    'categorical_label'
] = 0  # failure
gdf_target.loc[
    (gdf_target['date_created_parsed'] <= REFERENCE_DATE - timedelta(days=OBSERVATION_WINDOW_DAYS)) & 
    (gdf_target['date_closed_parsed'].isna()),
    'categorical_label'
] = 1  # success

# Filter mature POIs
df_mature = gdf_target[gdf_target['categorical_label'] != 2].copy()

print(f"Mature {TARGET_CATEGORY}s: {len(df_mature):,}")
print(f"  Failures: {(df_mature['categorical_label'] == 0).sum():,}")
print(f"  Successes: {(df_mature['categorical_label'] == 1).sum():,}")
print(f"  Failure rate: {(df_mature['categorical_label'] == 0).sum() / len(df_mature) * 100:.1f}%")

## Feature Extraction: Competition Features

Extract multiple distance scales (1km, 2km, 5km)

In [ ]:
print("="*80)
print("EXTRACTING COMPETITION FEATURES")
print("="*80)
print()

# Build spatial index for target category
target_tree = STRtree(gdf_target.geometry)

BUFFER_SIZES = [1000, 2000, 5000]  # meters

for buffer_m in BUFFER_SIZES:
    print(f"\nExtracting at {buffer_m}m...")
    
    counts = []
    for idx, poi in tqdm(df_mature.iterrows(), total=len(df_mature), desc=f"  {buffer_m}m"):
        buffer = poi.geometry.buffer(buffer_m)
        nearby_indices = target_tree.query(buffer)
        nearby = gdf_target.iloc[nearby_indices]
        
        # Count competitors (exclude self)
        competitors = nearby[nearby.index != idx]
        competitors = competitors[competitors.geometry.within(buffer)]
        
        counts.append(len(competitors))
    
    col_name = f'competitors_{buffer_m}m'
    df_mature[col_name] = counts
    print(f"  ✓ {col_name}: mean={np.mean(counts):.1f}, median={np.median(counts):.0f}")

print(f"\n✓ Competition features extracted")

## Feature Extraction: Nearest Competitor

In [ ]:
print("\nExtracting: Nearest competitor distance...")

nearest_distances = []
for idx, poi in tqdm(df_mature.iterrows(), total=len(df_mature), desc="  Nearest competitor"):
    others = gdf_target[gdf_target.index != idx]
    if len(others) > 0:
        distances = others.geometry.distance(poi.geometry)
        nearest_distances.append(distances.min())
    else:
        nearest_distances.append(10000)

df_mature['nearest_competitor_m'] = nearest_distances
print(f"✓ nearest_competitor_m: mean={np.mean(nearest_distances):.0f}m")

## Feature Extraction: Anchor POIs (Malls, Offices, Transport)

In [ ]:
print("="*80)
print("EXTRACTING ANCHOR POI FEATURES")
print("="*80)
print()

# Define anchor types
ANCHOR_TYPES = {
    'mall': ['mall'],
    'office': ['office'],
    'transport': ['transport']
}

for anchor_name, poi_types in ANCHOR_TYPES.items():
    print(f"\nExtracting: {anchor_name} counts...")
    
    # Filter anchor POIs
    gdf_anchor = gdf_all[gdf_all['poi_type'].isin(poi_types)]
    print(f"  {anchor_name} POIs: {len(gdf_anchor):,}")
    
    if len(gdf_anchor) == 0:
        # No POIs of this type
        df_mature[f'{anchor_name}_count_1000m'] = 0
        continue
    
    # Build spatial index
    anchor_tree = STRtree(gdf_anchor.geometry)
    
    counts = []
    for idx, poi in tqdm(df_mature.iterrows(), total=len(df_mature), desc=f"  {anchor_name}"):
        buffer = poi.geometry.buffer(1000)
        nearby_indices = anchor_tree.query(buffer)
        nearby = gdf_anchor.iloc[nearby_indices]
        within = nearby[nearby.geometry.within(buffer)]
        counts.append(len(within))
    
    col_name = f'{anchor_name}_count_1000m'
    df_mature[col_name] = counts
    print(f"  ✓ {col_name}: mean={np.mean(counts):.1f}")

print(f"\n✓ Anchor features extracted")

## Feature Extraction: Other POI Categories

In [ ]:
print("="*80)
print("EXTRACTING OTHER POI CATEGORY FEATURES")
print("="*80)
print()

# Define other relevant categories
OTHER_TYPES = ['residential', 'school', 'hospital', 'bank', 'gym', 'university']

for poi_type in OTHER_TYPES:
    print(f"\nExtracting: {poi_type} counts...")
    
    # Filter POIs
    gdf_type = gdf_all[gdf_all['poi_type'] == poi_type]
    print(f"  {poi_type} POIs: {len(gdf_type):,}")
    
    if len(gdf_type) == 0:
        df_mature[f'{poi_type}_count_1000m'] = 0
        continue
    
    # Build spatial index
    type_tree = STRtree(gdf_type.geometry)
    
    counts = []
    for idx, poi in tqdm(df_mature.iterrows(), total=len(df_mature), desc=f"  {poi_type}"):
        buffer = poi.geometry.buffer(1000)
        nearby_indices = type_tree.query(buffer)
        nearby = gdf_type.iloc[nearby_indices]
        within = nearby[nearby.geometry.within(buffer)]
        counts.append(len(within))
    
    col_name = f'{poi_type}_count_1000m'
    df_mature[col_name] = counts
    print(f"  ✓ {col_name}: mean={np.mean(counts):.1f}")

print(f"\n✓ Other POI features extracted")

## Feature Extraction: Demographics (District-Based)

In [ ]:
print("="*80)
print("EXTRACTING DEMOGRAPHIC FEATURES")
print("="*80)
print()

# Income estimates (millions IDR/month)
jakarta_income = {
    'Setiabudi': 22.8, 'Kebayoran Baru': 18.5, 'Menteng': 19.3,
    'Tanah Abang': 12.4, 'Cilandak': 16.2, 'Kebayoran Lama': 14.1,
    'Mampang Prapatan': 15.3, 'Tebet': 13.9, 'Pancoran': 11.7,
    'Pasar Minggu': 9.8, 'Jagakarsa': 9.5, 'Pesanggrahan': 9.2,
    'Gambir': 11.1, 'Kemayoran': 10.3, 'Sawah Besar': 9.1,
    'Senen': 8.1, 'Cempaka Putih': 10.8, 'Johar Baru': 8.9,
    'Cakung': 7.8, 'Jatinegara': 9.4, 'Kramat Jati': 8.5,
    'Matraman': 9.9, 'Pasar Rebo': 8.2, 'Ciracas': 7.9,
    'Cengkareng': 8.7, 'Grogol Petamburan': 11.5, 'Kalideres': 7.5,
    'Kebon Jeruk': 12.8, 'Kembangan': 8.4, 'Palmerah': 13.2,
}

# Population density (per km²)
jakarta_density = {
    'Cilandak': 5979, 'Jagakarsa': 12281, 'Kebayoran Baru': 7999,
    'Kebayoran Lama': 9629, 'Mampang Prapatan': 7112, 'Pancoran': 9885,
    'Pasar Minggu': 9081, 'Pesanggrahan': 7955, 'Setiabudi': 8572,
    'Tebet': 13010, 'Cempaka Putih': 8855, 'Gambir': 5746,
    'Johar Baru': 27135, 'Kemayoran': 14957, 'Menteng': 16111,
    'Sawah Besar': 27769, 'Senen': 19499, 'Tanah Abang': 17797,
    'Cakung': 11466, 'Ciracas': 10203, 'Jatinegara': 24500,
    'Kramat Jati': 12178, 'Matraman': 18670, 'Pasar Rebo': 11704,
    'Cengkareng': 13897, 'Kalideres': 15811, 'Kebon Jeruk': 12165,
    'Kembangan': 17094, 'Palmerah': 25872,
}

df_mature['income_district_m'] = df_mature['district'].map(
    lambda x: jakarta_income.get(str(x).replace(' ', ''), 10.5)
)

df_mature['density_district'] = df_mature['district'].map(
    lambda x: jakarta_density.get(str(x).replace(' ', ''), 12000)
)

print(f"✓ income_district_m: mean={df_mature['income_district_m'].mean():.1f}M IDR")
print(f"✓ density_district: mean={df_mature['density_district'].mean():.0f} per km²")

## Feature Extraction: Derived Features

In [ ]:
print("="*80)
print("CREATING DERIVED FEATURES")
print("="*80)
print()

# Demand-supply ratio
df_mature['demand_supply_1km'] = (
    df_mature['density_district'] / (df_mature['competitors_1000m'] + 1)
)
print(f"✓ demand_supply_1km: mean={df_mature['demand_supply_1km'].mean():.1f}")

# Income × Density interaction
df_mature['income_density_interaction'] = (
    df_mature['income_district_m'] * df_mature['density_district'] / 1000
)
print(f"✓ income_density_interaction: mean={df_mature['income_density_interaction'].mean():.1f}")

# Competition pressure (competitors per 1000 people)
df_mature['competition_pressure'] = (
    df_mature['competitors_1000m'] * 1000 / (df_mature['density_district'] + 1)
)
print(f"✓ competition_pressure: mean={df_mature['competition_pressure'].mean():.1f}")

print(f"\n✓ Derived features created")

## Summary: All Features Extracted

In [ ]:
# Get all feature columns
exclude_cols = [
    'name', 'latitude', 'longitude', 'date_created', 'date_closed',
    'district', 'regency', 'province', 'poi_type', 'source',
    'date_created_parsed', 'date_closed_parsed', 'event', 'duration',
    'categorical_label', 'geometry'
]

feature_cols = [col for col in df_mature.columns if col not in exclude_cols]
feature_cols = [col for col in feature_cols if df_mature[col].dtype in ['int64', 'float64']]

print(f"="*80)
print(f"TOTAL FEATURES EXTRACTED: {len(feature_cols)}")
print(f"="*80)
print()
print("Features:")
for i, feat in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {feat}")

## Train Gradient Boosting for Feature Importance

In [ ]:
print("="*80)
print("TRAINING GRADIENT BOOSTING (FOR FEATURE IMPORTANCE)")
print("="*80)
print()

# Prepare data
X = df_mature[feature_cols].fillna(0).replace([np.inf, -np.inf], 0).values
y = np.array(
    list(zip(df_mature['event'], df_mature['duration'])),
    dtype=[('event', bool), ('duration', float)]
)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=df_mature['event']
)

# Class weights
n_failures = y_train['event'].sum()
n_successes = len(y_train) - n_failures
sample_weights = np.where(
    y_train['event'],
    len(y_train) / (2 * n_failures),
    len(y_train) / (2 * n_successes)
)

print(f"Training data: {len(X_train):,} samples")
print(f"Features: {len(feature_cols)}")
print(f"Model: Gradient Boosting with {GB_CONFIG['n_estimators']} trees")
print(f"\n⚠ Training for feature importance ONLY (not accuracy)\n")

# Train
import time
gb_model = GradientBoostingSurvivalAnalysis(**GB_CONFIG)

start = time.time()
gb_model.fit(X_train, y_train, sample_weight=sample_weights)
elapsed = time.time() - start

print(f"\n✓ Training complete: {elapsed:.0f}s ({elapsed/60:.1f} min)")

## Feature Importance Analysis

In [ ]:
print("="*80)
print("FEATURE IMPORTANCE RANKING")
print("="*80)
print()

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Calculate cumulative importance
feature_importance['cumulative_importance'] = feature_importance['importance'].cumsum()
feature_importance['importance_pct'] = feature_importance['importance'] * 100
feature_importance['cumulative_pct'] = feature_importance['cumulative_importance'] * 100

print("\nTop 20 Most Important Features:\n")
print(feature_importance.head(20)[['feature', 'importance_pct', 'cumulative_pct']].to_string(index=False))

# Save full ranking
feature_importance.to_csv(OUTPUT_DIR / 'feature_importance_ranking.csv', index=False)
print(f"\n✓ Full ranking saved: {OUTPUT_DIR / 'feature_importance_ranking.csv'}")

## Visualize Top Features

In [ ]:
# Plot top 15 features
plt.figure(figsize=(12, 8))
top_15 = feature_importance.head(15)

plt.barh(top_15['feature'], top_15['importance_pct'], color='#2E86AB')
plt.xlabel('Importance (%)', fontsize=12, fontweight='bold')
plt.title(f'Top 15 Features - {TARGET_CATEGORY.capitalize()} Survival\n(Gradient Boosting Feature Importance)', 
          fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()

# Save
plt.savefig(OUTPUT_DIR / 'feature_importance_top15.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Chart saved: {OUTPUT_DIR / 'feature_importance_top15.png'}")

## Recommendations

In [ ]:
print("="*80)
print("RECOMMENDATIONS")
print("="*80)
print()

# Find features that contribute 80% of importance
top_80_pct = feature_importance[feature_importance['cumulative_pct'] <= 80]
print(f"Features contributing 80% of importance: {len(top_80_pct)}")
print(f"\nThese {len(top_80_pct)} features:\n")
for idx, row in top_80_pct.iterrows():
    print(f"  • {row['feature']:30s} {row['importance_pct']:5.1f}%")

print(f"\n" + "="*80)
print(f"NEXT STEPS")
print("="*80)
print()
print(f"1. Use ONLY the top {len(top_80_pct)} features for final model")
print(f"2. Discard the remaining {len(feature_cols) - len(top_80_pct)} features (contribute <20%)")
print(f"3. Train Random Survival Forest with these {len(top_80_pct)} features")
print(f"4. Optimize hyperparameters for accuracy")
print()
print(f"Expected benefit:")
print(f"  • {len(top_80_pct)/len(feature_cols)*100:.0f}% fewer features = faster training")
print(f"  • Less overfitting risk")
print(f"  • Same or better accuracy (removed noise)")

## Summary

This notebook extracted ALL candidate features and used Gradient Boosting to identify which ones actually matter.

**Output:**
- `feature_importance_ranking.csv` - Full ranking of all features
- `feature_importance_top15.png` - Visualization of top features
- Recommendation: Keep only top N features (80% cumulative importance)

**NOT for accuracy - FOR FEATURE SELECTION!**

---

*Generated with Claude Code*